# Provisioning AIO Cluster with K3D

This guide walks you through the steps to provision an Azure IoT Operations (AIO) cluster using the provided Bicep template. The Bicep template is based on the example available in an Azure IoT Operations repository.

> For this notebook, when running for the first time, you may need to set the kernel. To do this click on  `Select Kernel` in the top right corner of the notebook. Click `Jupyter Kernel...` then `Bash` as the kernel.

## Azure Resource Dependencies

1. Azure Key Vault
1. Azure Arc
1. Service Principal

## Prerequisites

1. A running GitHub Codespace of this repository.
1. Dev container setup with k3d cluster. Upon creation of the Codespace, a k3d cluster "devcluster" is already running and ready to use.
1. **Azure Subscription**: `az login` has been executed and default subscription has been set. Ensure you have access to an Azure subscription where you can deploy resources.

If required, reset your environment using the [00-clean-up.sh](./00-clean-up.sh) script, which will delete the k3d cluster and then recreate the cluster with the right image and settings. 

In [2]:
./00-clean-up.sh

Azure resource group 'rg-aio-86867' is being deleted
K3D registry deleted
INFO[0000] Deleting cluster 'devcluster'                
INFO[0002] Deleting cluster network 'k3d-devcluster'    
INFO[0002] Deleting 1 attached volumes...               
INFO[0002] Removing cluster details from default kubeconfig... 
INFO[0002] Removing standalone kubeconfig file (if there is one)... 
INFO[0002] Successfully deleted cluster devcluster!     
INFO[0000] Creating node 'k3d-devregistry.localhost'    
INFO[0000] Successfully created registry 'k3d-devregistry.localhost' 
INFO[0000] Starting Node 'k3d-devregistry.localhost'    
INFO[0000] Successfully created registry 'k3d-devregistry.localhost' 
# You can now use the registry like this (example):
# 1. create a new cluster that uses this registry
k3d cluster create --registry-use k3d-devregistry.localhost:5500

# 2. tag an existing local image to be pushed to the registry
docker tag nginx:latest k3d-devregistry.localhost:5500/mynginx:v0.1

# 3. push th

## Steps

### 1. Connect to Azure Arc
Set the params as env variables. Ensure that for the parameter `LOCATION` you use one of the [supported regions](https://learn.microsoft.com/en-us/azure/iot-operations/get-started/quickstart-deploy?tabs=linux#connect-a-kubernetes-cluster-to-azure-arc).


In [3]:
export SUFFIX=$(date +%s | cut -c6-10)
export RESOURCE_GROUP="rg-aio-$SUFFIX"
export CLUSTER_NAME="mycluster"
export LOCATION="westus2"
export APP_NAME="sp-aio-$SUFFIX"

Run the following script to connect the k3d Kubernetes cluster to Azure Arc.

In [4]:
./01-arc-connect.sh

ARC_CUSTOMLOCATION_OID is not set. Setting it to the value of LOCATION westus2.


{
  "id": "/subscriptions/20d5770a-8bb2-402f-9ade-11d91cb478a2/resourceGroups/rg-aio-87554",
  "location": "westus2",
  "managedBy": null,
  "name": "rg-aio-87554",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}
  "id": "/subscriptions/20d5770a-8bb2-402f-9ade-11d91cb478a2/resourceGroups/rg-aio-87554",
  "location": "westus2",
  "managedBy": null,
  "name": "rg-aio-87554",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}
E0311 20:06:13.332738  246387 memcache.go:287] couldn't get resource list for metrics.k8s.io/v1beta1: the server is currently unable to handle the request
E0311 20:06:13.334649  246387 memcache.go:121] couldn't get resource list for metrics.k8s.io/v1beta1: the server is currently unable to handle the request
E0311 20:06:13.338373  246387 memcache.go:121] couldn't get resource list for metrics.k8s.io/v1beta1: the server

### 2. Create service principal
Run the script to create a service principal. The script will save the app ID, client secret, and object Id of the created service principal in `~/.azure/servicePrincipal.json`.

In [5]:
./02-create-sp.sh $APP_NAME

Creating Service Principal ...


Set the following service principal variables for the next step.

In [ ]:
export AKV_SP_CLIENT_ID=$(cat ~/.azure/servicePrincipal.json | jq -r '.clientId')
export AKV_SP_CLIENT_SECRET=$(cat ~/.azure/servicePrincipal.json | jq -r '.clientSecret')
export AKV_SP_OBJECT_ID=$(az ad sp show --id $AKV_SP_CLIENT_ID --query id -o tsv)

### 3. Deploy the Bicep Template

Run the following script to provision the Azure Key Vault and deploy AIO with MQ. 

The script automatically sets the Azure Key Vault name based on the Resource Group name you set to `$RESOURCE_GROUP` variable and a random number to prevent name conflicts.

> **Note**: The deployment will take a few minutes to complete. To see the output of the `03-aio-deploy-core.sh` script, look in the `output.log` file created in the same directory.

In [ ]:
# Deploys the extensions and AIO core components
./03-aio-deploy-core.sh > output.log 2>&1 && tail -n 1 output.log

Finished deploying AIO Core components


In [ ]:
# Deploys the AIO CRDs
./04-aio-deploy-bicep.sh

Deployment Name: aio-deployment-28119
Check if AKV extension is installed
NAME                                                        READY   STATUS      RESTARTS        AGE
coredns-75fc8f8fff-7zvwf                                    1/1     Running     0               23m
local-path-provisioner-5b5579c644-xksm7                     1/1     Running     0               23m
helm-install-traefik-crd-jlc9d                              0/1     Completed   0               23m
helm-install-traefik-k568p                                  0/1     Completed   1               23m
svclb-traefik-ff87eaba-kq76m                                2/2     Running     0               23m
metrics-server-5c8978b444-rjhvv                             1/1     Running     0               23m
traefik-9c6dc6686-qqrvq                                     1/1     Running     0               23m
akvsecretsprovider-csi-secrets-store-provider-azure-mss22   1/1     Running     0               2m30s
secrets-store-csi-driver

{\ Finished .. ..
  "id": "/subscriptions/20d5770a-8bb2-402f-9ade-11d91cb478a2/resourceGroups/rg-aio-84619/providers/Microsoft.Resources/deployments/aio-deployment-28119",
  "location": null,
  "name": "aio-deployment-28119",
  "properties": {
    "correlationId": "008667c3-b981-44ae-b934-7179f9d0daac",
    "debugSetting": null,
    "dependencies": [],
    "duration": "PT5M46.5687573S",
    "error": null,
    "mode": "Incremental",
    "onErrorDeployment": null,
    "outputResources": [
      {
        "id": "/subscriptions/20d5770a-8bb2-402f-9ade-11d91cb478a2/resourceGroups/rg-aio-84619/providers/Microsoft.ExtendedLocation/customLocations/mycluster-cl",
        "resourceGroup": "rg-aio-84619"
      },
      {
        "id": "/subscriptions/20d5770a-8bb2-402f-9ade-11d91cb478a2/resourceGroups/rg-aio-84619/providers/Microsoft.ExtendedLocation/customLocations/mycluster-cl/resourceSyncRules/mycluster-cl-adr-sync",
        "resourceGroup": "rg-aio-84619"
      },
      {
        "id": "/subs

In [ ]:
# Deploys the AIO MQTT Broker
./05-aio-deploy-manifests.sh

Context "k3d-devcluster" modified.
issuer.cert-manager.io/mq-dmqtt-frontend created
broker.mq.iotoperations.azure.com/mq-instance-broker created
diagnosticservice.mq.iotoperations.azure.com/mq-instance-diagnostics-service created
brokerlistener.mq.iotoperations.azure.com/mq-tls-listener created
brokerlistener.mq.iotoperations.azure.com/az-mqtt-non-tls-listener created
NAME                 MODE          FRONTENDS   BACKEND CHAINS   MEMORY PROFILE   AGE   STATUS   STATUS DESCRIPTION
mq-instance-broker   distributed   2           2                tiny             0s             
Checking for running status of broker named 'mq-instance-broker'
broker.mq.iotoperations.azure.com/mq-instance-broker condition met
Checking for provisioning status of data processor instance named 'mycluster-processor'
instance.dataprocessor.iotoperations.azure.com/mycluster-processor condition met
Finished deploying AIO


### 4. Verify Deployment

Once the deployment is complete, you can verify the following k8s extensions in your cluster are in the Azure Portal with the status `Succeeded`:

- azure-iot-operations
- mq

You can also run the following script to validate the extensions were installed successfully.

In [ ]:
./07-validate-extensions.sh

### 5. Setup the observability stack (optional)

The observability stack is a set of Azure resources including Azure Monitor, Grafana, and Prometheus connections to the cluster to gather cluster insights, traces and metrics. Run the following script from the provisioning folder:

In [ ]:
./06-observability.sh

== Enabling observability ==
Command group 'config' is experimental and under development. Reference and support levels: https://aka.ms/CLI_refstatus
Creating an Azure Monitor Logs Analytics Workspace ...
{
  "accountId": "8a0789fd-29ae-4787-b9a7-7e6f24fe9329",
  "defaultIngestionSettings": {
    "dataCollectionEndpointResourceId": "/subscriptions/20d5770a-8bb2-402f-9ade-11d91cb478a2/resourceGroups/MA_monitor-mycluster_westus2_managed/providers/Microsoft.Insights/dataCollectionEndpoints/monitor-mycluster",
    "dataCollectionRuleResourceId": "/subscriptions/20d5770a-8bb2-402f-9ade-11d91cb478a2/resourceGroups/MA_monitor-mycluster_westus2_managed/providers/Microsoft.Insights/dataCollectionRules/monitor-mycluster"
  },
  "etag": "\"bf02ec8c-0000-0800-0000-65ef495e0000\"",
  "id": "/subscriptions/20d5770a-8bb2-402f-9ade-11d91cb478a2/resourcegroups/rg-aio-031124/providers/microsoft.monitor/accounts/monitor-mycluster",
  "location": "westus2",
  "metrics": {
    "internalId": "mac_8a0789fd-2

For more information about using the observability stack, go to [OBSERVABILITY.md](../../docs/OBSERVABILITY.md).

### 6. Testing MQ Broker

1. Open the MQTT UI in a terminal:

   ```bash
   mqttui
   ```

1. Open a new Bash terminal and publish a message.

In [ ]:
mqttui publish "in-cluster-topic" 'hello from within the pod'

3. Verify you have received the published message in the first terminal you opened the MQTT UI. For that, use the MQTT UI navigation instructions to navigate to the `in-cluster-topic`.

    ![in-cluster-topic in MQTTUI](../../docs/assets/mqttui-in-cluster-topic.png)

### 7. OPCUA Messages Simulator

You can provision the simulator into the system with:

In [ ]:
#./08-simulator.sh

This will create a pod (data-simulator) that will constantly run and send messages every 5 seconds with the two payloads of Good and MachineStatus.

You can open MQTTUI to visualize all the messages flowing in this order:

1. opcua-simulator
1. opcua
1. input
1. input/valid or input/invalid
1. current-shift-total
1. metrics/aio/machine-status and metrics/aio/total-count

![MQTTUI All Topics](../../docs/assets/mqttui-alltopics.png)


## Next Steps

Deploy the data processing pipelines in [infra/deployment](../deployment/DEPLOYMENT.ipynb)